In [1]:
# Important Imports/Functions
from collections import Counter
from hashlib import md5
from multiprocessing import Lock
import os
from pathlib import Path
import random
import tempfile
import time
import yaml
import pickle
import copy

import envoy
from joblib import Memory
from sklearn.base import BaseEstimator
from tqdm import tqdm

import numpy as np

# Generate two directories of 3000 tagsets: one that has been "filtered" with tag_punc, the other normal
def get_dirty_csids():
    with (PROJECT_ROOT / 'changeset_sets' /
      'threek_dirty_chunks.p').open('rb') as f:
        threeks = pickle.load(f)
    return threeks

def get_changeset(csid, iterative=False):
    changeset = None
    cs_name = ""
    if str(csid) in {'5', '6', '7'}:
        # Dirty fix for finger, autotrace
        globstr = '*[!16].5'
    else:
        globstr = '*.{}'.format(csid)
    if iterative:
        globstr += '.yaml'
    else:
        globstr += '.[!y]*'
    print(globstr)
    for csfile in CHANGESET_ROOT.glob(globstr):
        if changeset is not None:
            raise IOError(
                "Too many changesets match the csid {}, globstr {}".format(
                    csid, globstr))
        with csfile.open('r') as f:
            cs_name = str(csfile)
            changeset = yaml.load(f)
    if changeset is None:
        raise IOError("No changesets match the csid {}".format(csid))
    if 'changes' not in changeset or (
            'label' not in changeset and 'labels' not in changeset):
        logging.error("Malformed changeset, id: %d, changeset: %s",
                      csid, csfile)
        raise IOError("Couldn't read changeset")
    return changeset, cs_name


In [4]:
# First, must create a directory to hold 10,000 corresponding changesets. Then can just use tagset_gen.py
PROJECT_ROOT = Path('~/praxi').expanduser() # leave as project root to access any necessary files
CHANGESET_ROOT = Path('~/caches/changesets/').expanduser()

def parse_csids(csids, multilabel=False, iterative=False):
    """ Returns labels and features from csids, features are file sets
    file sets: list of string of format '644 /usr/.../file' """
    changesets = []
    changeset_names = []
    for csid in tqdm(csids):
        changeset, changeset_name = get_changeset(csid, iterative=iterative)
        changesets.append(changeset)
        changeset_names.append(changeset_name)
    return changesets, changeset_names


dirty_chunks = get_dirty_csids()
all_ids = dirty_chunks[0] + dirty_chunks[1] + dirty_chunks[2]
changesets, changeset_names = parse_csids(all_ids)

orig_tag_dir = '/home/ubuntu/praxi/tag_improvements/orig_tags'
edited_tag_dir = '/home/ubuntu/praxi/tag_improvements/edited_tags'



  0%|          | 0/3000 [00:00<?, ?it/s]

*.21043.[!y]*


  0%|          | 1/3000 [00:00<41:01,  1.22it/s]

*.19653.[!y]*


  0%|          | 2/3000 [00:01<33:21,  1.50it/s]

...

100%|█████████▉| 2997/3000 [21:50<00:00,  4.54it/s]

*.27038.[!y]*
*.21758.[!y]*


100%|█████████▉| 2999/3000 [21:50<00:00,  5.51it/s]

*.22653.[!y]*
*.102553.[!y]*


100%|██████████| 3000/3000 [21:52<00:00,  1.25it/s]


In [9]:
full_names = []
for name in changeset_names:
    cut_name = name.split('/')
    #print(cut_name[-1])
    name_w_path = '/home/ubuntu/praxi/caches/changesets/' + cut_name[-1]
    full_names.append(name_w_path)
    
print(full_names[0])

# now resave all changesets to orig_tag_dir 
#for file_name in full_names:
#    if(os.path.isfile(file_name)):
#        shutil.copy(filename, orig_tag_dir)

/home/ubuntu/praxi/caches/changesets/dwz.21043.rp.ubx.ts.yaml


In [10]:
print(full_names[0])

/home/ubuntu/praxi/caches/changesets/dwz.21043.rp.ubx.ts.yaml


In [13]:
import shutil
# now resave all changesets to orig_tag_dir 
for file_name in full_names:
    if(os.path.isfile(file_name)):
        print("isfile")
        shutil.copy(file_name, orig_tag_dir)

isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile

isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile

isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile
isfile

In [14]:
tag1 = 'zsh'
tag2 = '_zsh'
tag3 = 'zsh-'

def simple_tag(tag):
    """ Given a single tag, remove punctuations (or punctuation and numbers?)"""
    ntag = tag
    if tag[-1] in '[!@#$._-]':
        ntag = ntag[:-1]
    if tag[0] in '[!@#$._-]':
        ntag = ntag[1:]
    return ntag

print(simple_tag(tag1))
print(simple_tag(tag2))
print(simple_tag(tag3))

zsh
zsh
zsh
